# Export d’articles MDB

Ce notebook parcourt un export HTML de MDB et le transforme en un tableau d’articles. On peut transformer ce tableau en CSV ou json par exemple.

In [23]:
from bs4 import BeautifulSoup
from glob import glob
import pandas as pd
import re
import gender_guesser.detector as gender
from nameparser import HumanName
import unicodedata

In [2]:
pd.set_option('display.max_colwidth', 999)

In [3]:
files = glob('stories/*.html')

In [4]:
def getContents(file):
    with open(file, 'r') as f:
        doc = BeautifulSoup(f.read(), 'html.parser')
    meta = {}
    for tr in doc.select('table tr'):
        meta[tr.find('th').text.replace(':', '').strip()] = tr.find('td').text.strip()
    meta['Paragraphs'] = [i.text for i in doc.select('p') if i != '']
    return meta

In [5]:
cols = ['Titre', 'Chapeau', 'Pour Correction (LA)', 'Paragraphs']
df = pd.DataFrame(columns=cols)
for file in files:
    data = getContents(file)
    story = {}
    for key in cols:
        if key in data.keys():
            story[key] = data[key]
        else:
            print('WARNING: no', key, 'for story', data['Titre'])
            story[key] = ''
    df = df.append(pd.DataFrame([story], columns=cols))
print('Dataframe contains', len(df), 'stories.')

Dataframe contains 102 stories.


In [6]:
df.reset_index(inplace=True)

In [7]:
def getText(paragraph):
    p = [i.strip() for i in paragraph[1:-1]]
    return "\n".join(p).strip()

In [8]:
df['Text'] = df['Paragraphs'].apply(getText)

In [9]:
del df['Paragraphs']

In [10]:
d = gender.Detector()

def getGender(name):
    firstName = name.split(' ')[0]
    return d.get_gender(firstName)

def getId(titre):
    try:
        cardId = re.search('(trait |tarit)(\d{1,2})', titre)
        return cardId.group(2)
    except:
        return ''

def getName(titre):
    name = re.search('\d (27.09 ){0,1}([a-zA-ZÀ-ÿ’\- ]*)', titre)
    return name.group(2)

def getAge(titre):
    age = re.search('\d{2}[  ]ans', titre)
    if age:
        return age.group()
    else:
        return ''

df['cardId'] = df['Titre'].apply(getId)
df['Name'] = df['Titre'].apply(getName)
df['Age'] = df['Titre'].apply(getAge)
df['Gender'] = df['Name'].apply(getGender)

In [11]:
df = df[df['cardId'] != ''].copy()

In [12]:
df['Gender'].value_counts()

male           68
female         25
unknown         6
mostly_male     1
Name: Gender, dtype: int64

In [15]:
df[-df['Age'].str.contains('ans')]

,index,Titre,Chapeau,Pour Correction (LA),Text,cardId,Name,Age,Gender
48,0,Digital Shapers Portrait 42. 27.09 Toni Schneider,"Partenaire de True Ventures, San Francisco",Corrigé,"Pour les Suisses de la Silicon Valley, Toni Schneider est un vétéran: en 1989 déjà, il déménage de Meilen (ZH) pour étudier l’informatique en Californie. Il n’est plus rentré au pays: il a fondé deux start-up avec un succès mitigé (une en faillite, deux vendues), travaillé chez Yahoo! puis comme CEO chez Automattic, et développé le logiciel Wordpress, gestionnaire mondial de la majorité des blogs. Mais, à 48 ans, c’est surtout dans le capital-risque que Toni Schneider a du succès.\nTrue Ventures (20 employés, 8 partenaires) a investi dans 300 start-up, dont près de la moitié encore en portefeuille. «Nous avons réalisé quelques coups ces douze derniers mois», affirme Toni Schneider. Duo Security, par exemple, a été reprise par Cisco pour un montant en milliards. Domiciliée dans le Michigan, l’entreprise a été «l’un des premiers exemples qu’il n’est pas nécessaire de déménager dans la Silicon Valley pour lancer une start-up technologique; depuis, c’est devenu une tendance», dit Toni...",42,Toni Schneider,,male
95,0,"Digital Shapers Portrait 94. 27.09 Grace Torrellas, âge????????","Evangéliste de la blockchain, Lausanne",Corrigé,"Blockchain par-ci, blockchain par-là, blockchain partout. La technologie promet de tout rendre plus rapide, plus efficace et plus transparent, ce qui entre-temps est devenu une préoccupation des groupes internationaux. Un aspect est souvent oublié, dans tout ce battage sur la blockchain: l’impact social. Mais aux yeux de Grace Torrellas, la technologie de la blockchain sert surtout à rendre le monde «meilleur». Elle offre l’outil parfait pour suivre de près l’état des dons: avec Giveth.io, Grace Torrellas veut atteindre un nouveau niveau de transparence et de responsabilité pour les dons par le biais de la blockchain Ethereum.\nAvec sa dernière initiative «Blockchain 4 Humanity», la diplômée de Stanford et de l’EPFL veut avec un programme de soutien de 12 mois encourager d’autres start-up à faire le bien à l’aide de la blockchain. Grace Torrellas ne rate donc jamais une occasion de promouvoir le potentiel de la technologie. Elle est en contact avec la start-up fintech Nexussquared...",94,Grace Torrellas,,female


In [13]:
df[['Name', 'Gender', 'Age', 'Titre']]

,Name,Gender,Age,Titre
0,Solange Ghernaouti,female,59 ans,"Digital Shaper Portrait 22. 27.09 Solange Ghernaouti, 59 ans"
1,Klaus Hommels,male,51 ans,"Digital Shaper Portrait 29. 27.09 Klaus Hommels, 51 ans"
2,Caecilia Charbonnier,unknown,37 ans,"Digital Shaper Portrait 30. 27.09 Caecilia Charbonnier, 37 ans"
3,Patrick Warnking,male,50 ans,"Digital Shapers. Portrait 11. 27.09 Patrick Warnking, 50 ans"
4,Pedro Bados,male,38 ans,"Digital Shapers. Portrait 12.27.09 Pedro Bados, 38 ans"
5,Raffaello D’Andrea,male,51 ans,"Digital Shapers. Portrait 17. 27.09 Raffaello D’Andrea, 51 ans"
6,Luca Maria Gambardella,male,56 ans,"Digital Shapers. Portrait 19. 27.09 Luca Maria Gambardella, 56 ans"
7,Estefania Tapias,female,30 ans,"Digital Shapers. Portrait 28. 27.09 Estefania Tapias, 30 ans"
8,Florian Schweitzer,male,44 ans,"Digital Shapers. Portrait 43, 27.09 Florian Schweitzer, 44 ans"
9,Marco Abele,male,49 ans,"Digital Shapers. Portrait 47. 27.09 Marco Abele, 49 ans"


In [16]:
def getGenderFilter(gender):
    if gender == 'female':
        return 'f-femme'
    elif gender == 'male':
        return 'f-homme'
    else:
        return ''

In [17]:
def getAgeCat(age):
    try:
        ageInt = int(age[:2])
    except:
        return ''
    if ageInt >= 40:
        return 'f-plus-40'
    else:
        return 'f-moins-40'

In [18]:
df['cardTitre'] = df['Name']
df['cardSubtitle'] = df['Chapeau']

In [19]:
df['cardInfo1'] = df['Age']
df['cardInfo2'] = df['Chapeau']
#df['cardSection1Title'] = 'Petite bio'


In [20]:
def getLastName(name):
    return HumanName(name).last

In [21]:
df['Last Name'] = df['Name'].apply(getLastName)

In [59]:
def getParameter(name):
    nfkd_form = unicodedata.normalize('NFKD', name)
    name = u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
    return name.replace(' ', '-').lower()
df['parameter'] = df['Name'].apply(getParameter)

In [60]:
df_pic = pd.read_csv('pictures.csv')

In [82]:
def getPic(name):
    if name == 'S' or name == '':
        return False
    result = df_pic[df_pic['file'].str.contains(name, flags=re.IGNORECASE)]
    if len(result) == 1:
        return result['path'].values[0].replace('img', '2018')
    return '2018/bados.jpg'

In [62]:
df['cardImage'] = df['Last Name'].apply(getPic)

In [63]:
df.sort_values('Last Name', inplace=True)

In [64]:
df['cardFilter1Tri'] = df['Age'].apply(getAgeCat)
df['cardFilter2Tri'] = '(romand / alémanique)'
df['cardFilter3Tri'] = df['Gender'].apply(getGenderFilter)
df['cardFilter4Tri'] = '(Les précurseurs / Les créatifs / …)'
df['cardFilter5Tri'] = '(secteur d’activité)'
df['cardFilter6Tri'] = '(Indépendant / Chercheur / PME / Grande entreprise)'

In [66]:
df.sort_values('Last Name').to_csv('stories.csv')

In [67]:
df['Début texte'] = df['Text'].apply(lambda x: x[:200])

In [68]:
df['cardFilter1Tri'] = df['Age'].apply(getAgeCat)
df['cardFilter2Tri'] = '(romand / alémanique)'
df['cardFilter3Tri'] = df['Gender'].apply(getGenderFilter)
df['cardFilter4Tri'] = '(Les précurseurs / Les créatifs / …)'
df['cardFilter5Tri'] = '(secteur d’activité)'
df['cardFilter6Tri'] = '(Indépendant / Chercheur / PME / Grande entreprise)'

In [69]:
df['cardSection5Text'] = '(liens Le Temps)'
df['cardSection6Text'] = '(profil linkedin)'

In [89]:
df['cardImage'] = df['cardImage'].apply(lambda x: x.split('/')[-1].replace(' ', '_') if isinstance(x, str) else x)

In [70]:
df[['cardId', 'Name', 'Gender', 'Age', 'Chapeau', 'Début texte', 'cardFilter1Tri',
       'cardFilter3Tri', 'cardFilter4Tri', 'cardFilter5Tri', 'cardFilter6Tri',
       'cardSection5Text', 'cardSection6Text']].to_csv('export_pour_etudiant.csv')

In [91]:
df['URL'] = df['parameter'].apply(lambda x: "https://labs.letemps.ch/interactive/2018/_digital-shapers/?portrait={}".format(x))

In [92]:
df[['cardId', 'Name', 'cardSubtitle', 'cardImage', 'Gender', 'Age', 'Début texte', 'cardFilter1Tri',
       'cardFilter3Tri', 'cardFilter4Tri', 'cardFilter5Tri', 'cardFilter6Tri',
       'cardSection5Text', 'cardSection6Text', 'URL']].to_excel('complement.xls')

In [72]:
df

,index,Titre,Chapeau,Pour Correction (LA),Text,cardId,Name,Age,Gender,cardTitre,...,cardFilter2Tri,cardFilter3Tri,cardFilter4Tri,cardFilter5Tri,cardFilter6Tri,cardSection2Text,Début texte,cardSection5Text,cardSection6Text,parameter
9,0,"Digital Shapers. Portrait 47. 27.09 Marco Abele, 49 ans","Fondateur et CEO de Tend, Zoug",Corrigé,"De responsable numérique au CS à entrepreneur de start-up: en octobre 2017, Marco Abele a troqué son costume sur mesure contre un jean et un t-shirt, et fondé la société Tend. La plateforme d’échange offre des expériences d’un type particulier: les clients peuvent, par exemple, réserver un voyage dans une Porsche 356A Speedster de 1955 ou faire une excursion privée dans une cave à vin en recevant 18 bouteilles de grands vins par an. Tend s’occupe de l’ensemble du pack zéro souci, moyennant un supplément bien sûr. Les clients peuvent payer ces expériences de luxe avec ce qu’on appelle des jetons, des avoirs numériques protégés par une blockchain.\n«Avec Tend, je veux atteindre un large public qui enrichit sa vie et vit ses rêves», explique l’Allemand de 49 ans. C’est le même désir qui l’a aidé à passer du monde du corporate au monde de la start-up. Les produits Tend ont été lancés cette année en Suisse. Pour 2019, Abele veut se développer au-delà des frontières helvétiques: «Dans q...",47,Marco Abele,49 ans,male,Marco Abele,...,(romand / alémanique),f-homme,(Les précurseurs / Les créatifs / …),(secteur d’activité),(Indépendant / Chercheur / PME / Grande entreprise),,"De responsable numérique au CS à entrepreneur de start-up: en octobre 2017, Marco Abele a troqué son costume sur mesure contre un jean et un t-shirt, et fondé la société Tend. La plateforme d’échange",(liens Le Temps),(profil linkedin),marco-abele
4,0,"Digital Shapers. Portrait 12.27.09 Pedro Bados, 38 ans","CEO et cofondateur de Nexthink, Lausanne",Corrigé,"«2017 a été une année incroyable. Nous avons gagné plus de 200 nouveaux clients», explique Pedro Bados. Aujourd’hui, plus de 850 clients utilisent les produits de l’entreprise lausannoise de logiciels, dont des gouvernements, des banques nationales et des entreprises comme Migros, Deloitte ou Amag. Nexthink examine attentivement les programmes de calculs de l’entreprise et analyse la qualité des services informatiques utilisés du point de vue de l’employé. L’idée de Nexthink est venue à Pedro Bados en 2004, avec ses camarades Vincent Bieri et Patrick Hertzog, lors d’un projet de recherche sur l’intelligence artificielle à l’EPFL. «Le département informatique fournit ses services à l’utilisateur final, sans disposer ni de la transparence nécessaire ni des bonnes informations pour déterminer si l’utilisateur utilise ces services et s’il en a besoin», explique Pedro Bados. Le Suisse a étudié l’électrotechnique et l’informatique à l’Université de Saragosse, en Espagne.",12,Pedro Bados,38 ans,male,Pedro Bados,...,(romand / alémanique),f-homme,(Les précurseurs / Les créatifs / …),(secteur d’activité),(Indépendant / Chercheur / PME / Grande entreprise),,"«2017 a été une année incroyable. Nous avons gagné plus de 200 nouveaux clients», explique Pedro Bados. Aujourd’hui, plus de 850 clients utilisent les produits de l’entreprise lausannoise de logiciels",(liens Le Temps),(profil linkedin),pedro-bados
92,0,"Digital Shapers Portrait 91. 27.09 Nick Beglinger, 48 ans","Cofondateur et CEO de Cleantech21, Zurich",Corrigé,"Jusqu’en 2016, Nick Beglinger a été président de l’association professionnelle Swisscleantech, consacrée à l’économie durable et issue en 2009 de la fondation à but non lucratif Cleantech21. Cette dernière est la principale occupation de Nick Beglinger, aujourd’hui âgé de 48 ans. «La durabilité doit devenir la norme», selon le site web de la fondation. Nick Beglinger veut construire un réseau d’entreprises actives au niveau mondial, pour mettre en œuvre à long terme les objectifs de l’Accord de Paris sur le climat. Il faut inciter le politique à mettre en œuvre les conditions-ca

In [ ]:
!open 